In [1]:
# load csv
import pandas as pd
csv_path = r"https://docs.google.com/spreadsheets/d/1EvIFLGpsKvuOgdX6F9MYlGl9rCLjbKHPlE8srBDMwYo/edit?usp=sharing"
url = csv_path.replace('/edit?usp=sharing', '/export?format=xlsx')

df = pd.read_excel(url, sheet_name="iPad")

print(df.size)
df.columns

4981


Index(['id', 'date', 'writer', 'model', 'screen_size', 'generation', 'ram',
       'storage', 'cellular', 'apple_care_plus', 'unused', 'price',
       'price for statistics', 'platform', 'url', 'title', 'content'],
      dtype='object')

In [2]:
import requests
item_csv_path = r"https://docs.google.com/spreadsheets/d/1eRkhdeDaD60IOajReYVRId28CM4IKQJiheL-yBWfzUw/edit?usp=sharing"
url = item_csv_path.replace('/edit?usp=sharing', '/export?format=xlsx')


item_df = pd.read_excel(url, sheet_name="iPad-Docs")

api_db = {}
reverse_db = {}

for index, row in item_df.iterrows():
    value = (row["item id"], row["option id"])
    key = (row["item"], row["gen"], row["cellular"], row["storage"])
    print(key)
    api_db[key] = value
    reverse_db[value] = key


('iPad Mini', '6세대', 'Wi-Fi', '64GB')
('iPad Mini', '6세대', 'Wi-Fi', '256GB')
('iPad Mini', '6세대', 'Wi-Fi+Cellular', '64GB')
('iPad Mini', '6세대', 'Wi-Fi+Cellular', '256GB')
('iPad Air', '4세대', 'Wi-Fi', '64GB')
('iPad Air', '4세대', 'Wi-Fi', '256GB')
('iPad Air', '4세대', 'Wi-Fi+Cellular', '64GB')
('iPad Air', '4세대', 'Wi-Fi+Cellular', '256GB')
('iPad Air', '5세대', 'Wi-Fi', '64GB')
('iPad Air', '5세대', 'Wi-Fi', '256GB')
('iPad Air', '5세대', 'Wi-Fi+Cellular', '64GB')
('iPad Air', '5세대', 'Wi-Fi+Cellular', '256GB')
('iPad', '8세대', 'Wi-Fi', '32GB')
('iPad', '8세대', 'Wi-Fi', '128GB')
('iPad', '8세대', 'Wi-Fi+Cellular', '32GB')
('iPad', '8세대', 'Wi-Fi+Cellular', '128GB')
('iPad', '9세대', 'Wi-Fi', '64GB')
('iPad', '9세대', 'Wi-Fi', '256GB')
('iPad', '9세대', 'Wi-Fi+Cellular', '64GB')
('iPad', '9세대', 'Wi-Fi+Cellular', '256GB')
('iPad', '10세대', 'Wi-Fi', '64GB')
('iPad', '10세대', 'Wi-Fi', '256GB')
('iPad', '10세대', 'Wi-Fi+Cellular', '64GB')
('iPad', '10세대', 'Wi-Fi+Cellular', '256GB')
('iPad Pro 11', '2세대', 'Wi-Fi', 

In [12]:
# API Docs를 기반으로 CSV 생성
import warnings
warnings.filterwarnings('ignore')

datas = []
total = 0
for key in api_db:
    model, gen, cellular, storage = key
    gen = gen[:-2] + "th"
    cellular = True if "Cellular" in cellular else False
    if "iPad Pro" in model:
        screen_size = float(model.split("iPad Pro")[1].strip())
        model = "iPad Pro"
        entity_list = df[(df["model"] == model) & (df["generation"] == gen) & (df["cellular"] == cellular) & (df["storage"] == storage) & (df["screen_size"] == screen_size)]
    else:
        screen_size = ""
        entity_list = df[(df["model"] == model) & (df["generation"] == gen) & (df["cellular"] == cellular) & (df["storage"] == storage)]

    total += entity_list.size
    item_id, option_id = api_db[key]
    for unopened in [True, False]:
        option_entity_list = entity_list[entity_list["unused"] == unopened]

        if option_entity_list.size <= 1:
            if option_entity_list.size == 0:
                print(reverse_db[(item_id, option_id)], end="")
                print(f"{'미개봉' if unopened else '중고'} , {'셀룰러' if cellular else '와이파이'}")
            datas.append({
                "item_id": item_id,
                "option_id": option_id,
                "start_date": "2023-04-30",
                "end_date": "2023-05-27",
                "unopened": unopened,
                "mean_std": round(option_entity_list["price for statistics"].mean()) if option_entity_list.size == 1 else 0,
                "mean_20": 0,
                "mean_80": 0,
            })
            continue

        # 평균
        mean = option_entity_list["price for statistics"].mean()

        # 중앙 값
        median = option_entity_list["price for statistics"].median()

        # 표준 편차
        std = option_entity_list["price for statistics"].std()

        # 평균으로부터 표준편차 이내의 값 평균
        mean_std = option_entity_list[(option_entity_list["price for statistics"] >= (mean - std)) & (option_entity_list["price for statistics"] <= (mean + std))]["price for statistics"].mean()

        # 상위 0 ~ 20% 사이 값들의 평균
        top_20_percentile = option_entity_list["price for statistics"].quantile(0.8)
        top_20_percent_values = option_entity_list[option_entity_list["price for statistics"] >= top_20_percentile]
        mean_20 = top_20_percent_values.mean()["price for statistics"]

        # 하위 20% 평균
        bottom_20_percentile = option_entity_list["price for statistics"].quantile(0.2)
        bottom_20_percent_values = option_entity_list[option_entity_list["price for statistics"] <= bottom_20_percentile]
        mean_80 = bottom_20_percent_values.mean()["price for statistics"]

        if mean_20 == mean_80:
            mean_std = mean_20
            mean_20 = mean_80 = 0

        datas.append({
            "item_id": item_id,
            "option_id": option_id,
            "start_date": "2023-04-30",
            "end_date": "2023-05-27",
            "unopened": unopened,
            "mean_std": round(mean_std),
            "mean_20": round(mean_20),
            "mean_80": round(mean_80),
        })


datas = pd.DataFrame(datas)
datas.to_csv("ipad-data.csv", index=False)


('iPad Mini', '6세대', 'Wi-Fi+Cellular', '256GB')미개봉 , 셀룰러
('iPad Air', '4세대', 'Wi-Fi', '64GB')미개봉 , 와이파이
('iPad Air', '4세대', 'Wi-Fi', '256GB')미개봉 , 와이파이
('iPad Air', '4세대', 'Wi-Fi', '256GB')중고 , 와이파이
('iPad Air', '4세대', 'Wi-Fi+Cellular', '64GB')미개봉 , 셀룰러
('iPad Air', '4세대', 'Wi-Fi+Cellular', '256GB')미개봉 , 셀룰러
('iPad Air', '4세대', 'Wi-Fi+Cellular', '256GB')중고 , 셀룰러
('iPad Air', '5세대', 'Wi-Fi+Cellular', '256GB')미개봉 , 셀룰러
('iPad Air', '5세대', 'Wi-Fi+Cellular', '256GB')중고 , 셀룰러
('iPad', '8세대', 'Wi-Fi', '32GB')미개봉 , 와이파이
('iPad', '8세대', 'Wi-Fi', '128GB')미개봉 , 와이파이
('iPad', '8세대', 'Wi-Fi+Cellular', '32GB')미개봉 , 셀룰러
('iPad', '8세대', 'Wi-Fi+Cellular', '128GB')미개봉 , 셀룰러
('iPad', '8세대', 'Wi-Fi+Cellular', '128GB')중고 , 셀룰러
('iPad', '9세대', 'Wi-Fi+Cellular', '64GB')미개봉 , 셀룰러
('iPad', '9세대', 'Wi-Fi+Cellular', '256GB')미개봉 , 셀룰러
('iPad', '9세대', 'Wi-Fi+Cellular', '256GB')중고 , 셀룰러
('iPad', '10세대', 'Wi-Fi+Cellular', '64GB')미개봉 , 셀룰러
('iPad', '10세대', 'Wi-Fi+Cellular', '64GB')중고 , 셀룰러
('iPad', '10세대', 'Wi-Fi+Ce

In [16]:
import time
# API Call
api_url_template = "https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/{}/option/{}"

# read csv
df = pd.read_csv("ipad-data.csv")

for index, row in df.iterrows():
    item_id = row["item_id"]
    option_id = row["option_id"]
    date = row["end_date"]
    mean = row["mean_std"]
    high = row["mean_20"]
    low = row["mean_80"]
    unopened = row["unopened"]


    api_url = api_url_template.format(item_id, option_id)
    if unopened:
        api_url += "?unopened=true"
    else:
        api_url += "?unopened=false"

    print(api_url)

    data = {
        "date": date,
        "mid": int(mean),
        "high": int(high),
        "low": int(low),
    }


    print(f"{data}")
    response = requests.post(api_url, json=data)
    print(response.status_code)
    print(response.text)
    print()
    time.sleep(2)


https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/6/option/1?unopened=true
{'date': '2023-05-27', 'mid': 620750, 'high': 630000, 'low': 610000}
200
{"_writeTime":{"_seconds":1686375154,"_nanoseconds":274657000}}

https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/6/option/1?unopened=false
{'date': '2023-05-27', 'mid': 520000, 'high': 620000, 'low': 452500}
200
{"_writeTime":{"_seconds":1686375156,"_nanoseconds":647910000}}

https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/6/option/2?unopened=true
{'date': '2023-05-27', 'mid': 735000, 'high': 780000, 'low': 690000}
200
{"_writeTime":{"_seconds":1686375158,"_nanoseconds":942475000}}

https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/6/option/2?unopened=false
{'date': '2023-05-27', 'mid': 656667, 'high': 850000, 'low': 500000}
200
{"_writeTime":{"_seconds":1686375161,"_nanoseconds":752498000}}

https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/6/opti